In [12]:
#Importing librarys

from bs4 import BeautifulSoup
import requests
import csv

import pandas as pd
import numpy as np

In [13]:
# Getting the HTML file and finding the table in it
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')


In [14]:
#creating the CSV file that will be used for saving the data
csv_file = open('csv_toronto_neighbourhood.csv', 'w')
csv_writer = csv.writer(csv_file)


In [15]:
#Writing the header in the CSV file
row = []

for th in table.find_all('th'):
    row_value = th.text.rstrip()
    
    row.append(row_value)
print(row)
csv_writer.writerow(row)

['Postcode', 'Borough', 'Neighbourhood']


32

In [16]:
# Write data to the CSV file
for tr in table.find_all('tr'):
    i=0
    row=[]
    discard = False
    postalCode = None
    borough = None
    neighbourhood = None
    for td in tr.find_all('td'):
        if i==0:
            postalCode = td.text.rstrip()
        elif i==1:
            if (td.text.rstrip() == 'Not assigned'):
                discard = True
            else:
                borough = td.text.rstrip()
        else:
            i=0
            if (td.text.rstrip() == 'Not assigned'):
                neighbourhood = borough
            else:
                neighbourhood = td.text.rstrip()
        i=i+1
    if (discard == False):
        row.append(postalCode)
        row.append(borough)
        row.append(neighbourhood)
        csv_writer.writerow(row)
 

In [17]:
#Closing the CSV File
csv_file.close()

In [18]:
#opening the CSV file
df = pd.read_csv('csv_toronto_neighbourhood.csv')
df.head()


,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park


In [19]:
#Knowing the shape of the CSV file
df.shape

(212, 3)

In [20]:
#Grouping the neighbourhoods with the same postal code
df_mod = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df_mod = df_mod.reset_index()
df_mod.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
#Knowing the new shape of the CSV file
df_mod.shape

(103, 3)